In [1]:
# NAME PARAMS
drive_folder = "drive/My Drive/[NN] Music Generation/"    # workspace folder in gdrive
dataset_path = "JazzDataset/"                             # dataset path
channel = "Piano"                                         # midi channel to extract

name = "test_name"                                        # model name
checkPoint_path = './checkpoint/weights_'+name+'.hdf5'    # path to save model
log_dir = './logs'                                        # path for Tensorboard logs

In [2]:
# TRAINING PARAMS
vocab_threshold = 50
seq_length = 32
batch_size = 64
n_epochs = 100
test_ratio = 0.05

# NET PARAMS
n_filters = 256
kernel_size = 2
n_resBlocks = 7

#Drive Access

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir(drive_folder)

# Libraries

In [5]:
from preprocessing import *
from models import simplifiedWaveNN
from postprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.callbacks import *

# Implementation

In [6]:
print("Loading MID dataset...")
mid_songs = load_dataset(dataset_path, channel)

Loading MID dataset...
Loading music from:  JazzDataset/Duke _ His Orchestra Ellington - Satin Doll 2.mid
Loading music from:  JazzDataset/Louis Armstrong - Sweet Adeline.mid
Loading music from:  JazzDataset/Ray_Charles_-_What_d_I_Say.mid
Loading music from:  JazzDataset/Fats Waller - Wish Upon A Star.mid
Loading music from:  JazzDataset/Ray_Charles_-_Georgia_On_My_Mind.mid
Loading music from:  JazzDataset/Reinhardt Django - After You`ve Gone.mid
Loading music from:  JazzDataset/Glenn _ His Orchestra Miller - Chattanooga Choo Choo 2.mid
Loading music from:  JazzDataset/Jazz Standardsdiversen - Lullaby Of Birdland.mid
Loading music from:  JazzDataset/Fitzgerald_Ella_Ray_Johnny_-_All_Of_Me.mid
Loading music from:  JazzDataset/Fats Waller - Tea For Two.mid
Loading music from:  JazzDataset/Duke _ His Orchestra Ellington - The Mooch.mid
Loading music from:  JazzDataset/Louis Armstrong - C`Est Si Bon.mid
Loading music from:  JazzDataset/Louis Prima - Sing Sing Sing.mid
Loading music from:  J

In [8]:
print("Loading CSV dataset...")
csv_songs = load_csv_dataset("./Jazz-midi.csv")
print("Extracted "+str(len(csv_songs))+" songs")

Loading CSV dataset...
Extracted 818 songs


In [9]:
songs = mid_songs #+ csv_songs[0:400] # for full dataset
#songs = mid_songs # only for JazzSet

In [10]:
print("Creating vocabularies...")
vocab, inverted_vocab, counts = make_vocabs(songs, vocab_threshold)
print("Vocabs size: "+str(len(vocab)))

Creating vocabularies...
Vocabs size: 108


In [11]:
print("Dataset preprocessing...")
x, y = dataset_preprocessing(songs, vocab, seq_length)
print("Dataset Shape:")
print("|->X: "+str(x.shape))
print("|->Y: "+str(y.shape))

Dataset preprocessing...
Dataset Shape:
|->X: (18111, 64, 1)
|->Y: (18111, 108)


In [12]:
print("Splitting dataset...")
train_x, dev_x, train_y, dev_y = train_test_split(x, y, test_size=test_ratio)
print("Training-Set Shape:")
print(train_x.shape)
print(train_y.shape)
print("Dev-Set Shape:")
print(dev_x.shape)
print(dev_y.shape)

Splitting dataset...
Training-Set Shape:
(17205, 64, 1)
(17205, 108)
Dev-Set Shape:
(906, 64, 1)
(906, 108)


In [13]:
out_size = len(vocab)
model = simplifiedWaveNN(seq_length, out_size)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 64, 128)      384         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 128)      512         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 128)      0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
y_train_index = remove_1hot(train_y)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_index),y_train_index)
keys = np.arange(len(vocab))
values = np.ones(len(vocab))
for i in range(len(class_weights)):
  values[i] = class_weights[i]
dict_loss = dict(zip(keys, values))
print(dict_loss)

In [ ]:
callbacks = [ModelCheckpoint(filepath = checkPoint_path, save_weights_only=True, monitor='loss', mode='min', save_best_only=True), TensorBoard(log_dir=log_dir) ]
history = model.fit(train_x, train_y, epochs=n_epochs, validation_data=(dev_x, dev_y), callbacks=callbacks, class_weight=dict_loss, shuffle=True)

In [ ]:
model = model.load_weights(checkPoint_path)
gen_song = generate_song(model_, inverted_vocab, 100, train_x, seq_length)
print("Generated song:\n",gen_song)
create_midi(gen_song, "Piano", "", name+".mid")
print("Song saved on drive.")